# AlgTest Process TPM 2.0 data visualisation

An example of TPM visualizations.

In [ ]:
#!python -m venv venv
#!source /home/tjaros/git/algtest-pyprocess/.venv/bin/activate.fish
#!python -m pip install -r ./requirements.txt

### Creation of the metadata.json

If you already have the `metadata.json` file you want to use, please skip this step

In [ ]:
#!python process.py tpm metadata-update /home/tjaros/storage/research/tpm/

### Load metadata

In [ ]:
import json

def load_metadata(metadata_path):
    try:
        metadata: ReportMetadata = {}
        with open(metadata_path, "r") as f:
            metadata = json.load(f)

        assert metadata
        entries = metadata["entries"].values()
        assert 0 < len(entries)
    except:
        print("report_create: retrieving metadata was unsuccessful")
        return {}

    # We now group entries by vendor
    grouped = {}
    for entry in entries:
        vendor = entry.get("vendor")
        if vendor is None:
            print(
                f"report_create: entry {entry} does not contain vendor")
            continue

        grouped.setdefault(vendor, [])
        grouped[vendor].append(entry)
    return grouped

metadata = load_metadata('./metadata.json')

### Load profiles from vendor

In [ ]:
from algtestprocess.modules.data.tpm.manager import TPMProfileManager
from algtestprocess.modules.data.tpm.enums import CryptoPropResultCategory as cat

cryptoprops = {}
for entry in metadata.get('INTC'):
    tpm_name = entry['TPM name']
    for measurement_path in entry['measurement paths']:
        man = TPMProfileManager(measurement_path)
        cryptoprops[tpm_name] = man.cryptoprops

### Merge the dataframes for same devices

In [ ]:
tpms = {}
for tpm_name, cpp in cryptoprops.items():
    if tpm_name not in tpms:
        tpms.setdefault(tpm_name, cpp)
    else:
        tpms[tpm_name] = tpms[tpm_name] + cpp

In [ ]:
import re
from math import inf

def tpm_sorted(profiles, device_name):
    """
    Sorts the profiles according to manufacturer id alphabetically, then
    firmware version numerically

    Assumes device name is in the form of rgx
    """
    RGX = r"(\s*.+)+\s\s*\d+(\.\d+)*(\s[\[]\d+[\]])?"
    try:
        assert all([re.match(RGX, device_name(p)) is not None for p in profiles])
    except AssertionError:
        print("These device names do not match format")
        print([name for p in profiles if not re.match(RGX, (name := device_name(p)))])

    def key_f(profile):
        manufacturer = version = idx = inf
        numbers = [inf] * 4
        l, r = device_name(profile).rsplit(maxsplit=1)

        if re.match(r"[\[]\d+[\]]", r):
            idx = int(r.replace("[", "").replace("]", ""))
            manufacturer, firmware = l.rsplit(maxsplit=1)
        else:
            manufacturer, firmware = l, r

        numbers = [int(x) for x in filter(None, firmware.split("."))]

        return [manufacturer] + numbers + [idx]

    return sorted(profiles, key=key_f)

def _table(l, cols, header):
    # header repeat col times
    out = ""
    out += "| " + (" | ".join(header) + " | ") * cols + "\n"
    out += "| " + ("|".join(["---"] * (cols * len(header))) + " | ") + "\n"

    i = 0
    while i < len(l):
        out += "| "
        for _ in range(cols):
            if i < len(l):
                entries = l[i]
            else:
                entries = [" " for _ in range(len(header))]

            assert len(entries) == len(header)

            out += " | ".join(map(str, entries)) + " | "
            i += 1
        out += "\n"
    return out

In [ ]:
from IPython.display import display, Markdown
# If you find false, it would be wise to check metadata for the corresponding files, as to 
# why it was not possible to load the cryptoprops
display(Markdown(_table(list(map(lambda x: [x[0], x[1] is not None], tpms.items())), 2, ['TPM Name', 'Profile object']))) 

In [ ]:
# Here is the sorted order of tpms, to polish according to actual versions u need 
# to sort them manually.
# NOTE: We sort only according to MANUFACTURER FIRMWARE_VERSION
tpms_sorted = tpm_sorted(list(filter(None, tpms.values())), lambda cpps: f"{cpps.manufacturer} {cpps.firmware_version}")
display(Markdown(_table(list(map(lambda x: [x[0], x[1].device_name, x[1].results.get(cat.RSA_1024) is not None, x[1].results.get(cat.RSA_2048) is not None], enumerate(tpms_sorted))), 1, ['Index', 'TPM Name', 'RSA 1024', 'RSA 2048']))) 

In [ ]:
tpms = tpms_sorted

In [ ]:
import matplotlib.pyplot as plt
from algtestprocess.modules.visualization.heatmap import Heatmap
import matplotlib.gridspec as gridspec

# Creating the figure with a constrained layout to avoid axes overlapping
fig = plt.figure(layout='constrained', figsize=(8.3, 11.7), dpi=800)
fig.get_layout_engine().set(w_pad=0, h_pad=0, hspace=0,
                            wspace=0)
# Creating a gridspec of size= (2,3)
ncols = 4
nrows = 6
GridSpec = gridspec.GridSpec(ncols=ncols, nrows=nrows, figure= fig)
subfig_count = ncols * nrows

def cpps2fig(cpps, fig):
    if cpps is None:
        return None

    result = cpps.results.get(cat.RSA_2048)
    if result is None:
        return None

    tpm_name = cpps.device_name
    df = result.data
    return Heatmap(df, tpm_name, fig=fig, ticks=False, legend=False, parts=['heatmap', 'text'], part_height_ratios=[1, 0.1], text_font_size = 9, label_values=False).build()

count = 0
row = 0
col = 0
for cpps in tpms:
    if count >= subfig_count:
        break
    
    subfig = fig.add_subfigure(GridSpec[row, col])
    if cpps2fig(cpps, subfig) is not None:
        if col + 1 >= ncols:
            row += 1
            col  = 0
        else:
            col += 1
        count += 1
    else:
        print(f"Plot for {cpps.device_name} failed")
    
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
# defining legend style and data
green_line = mlines.Line2D([], [], color='green', label='$P_{min}$')
blue_line = mlines.Line2D([], [], color='blue', label='$P_{max}$')
orange_line = mlines.Line2D([], [], color='orange', label='$Q_{min}$')
purple_line = mlines.Line2D([], [], color='purple', label='$Q_{max}$')
skyblue_line = mlines.Line2D([], [], color='skyblue', label='$P=Q$')

plt.legend(handles=[green_line, blue_line, orange_line, purple_line, skyblue_line], loc=(1.5,1), fontsize='x-large')
plt.show()